## Install `simple_salesforce`  module

Remove the comment symbol (#) to run `!pip install simple_salesforce`. Once this module is installed, you can comment it out again. 

In [ ]:
#!pip install simple_salesforce

## Import dependencies

In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()



## Import configuration variables

In [2]:
import os
# Import API key
orig_wd = os.getcwd()
os.chdir(os.path.join('..','..','..','..','..'))
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
os.chdir(orig_wd)

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [78]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [108]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Load Students

In [22]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [24]:
student_data['FullName'] = student_data['FirstName'].str.cat(student_data['LastName'], sep=" ")
student_data['Company'] = 'Dartling University'
#student_data = student_data
student_load_df = student_data[['LastName','FirstName','ID_Student','Company','Gender','MiddleName','StudentID','FullName']].\
    rename(columns={'FullName':'Name',
                    'LastName':'LastName__c',
                    'FirstName':'FirstName__c',
                    'Company':'Company__c',
                    'Gender':'Gender__c',
                    'ID_Student':'ID_Student__c',
                    'MiddleName':'MiddleName__c',
                    'StudentID':'StudentID__c'})
                  
student_load_df.head()

,LastName__c,FirstName__c,ID_Student__c,Company__c,Gender__c,MiddleName__c,StudentID__c,Name
0,Moore,Heather,33,Dartling University,F,Alice,25004961,Heather Moore
1,Multak,Ilana,34,Dartling University,F,Cecille,25003514,Ilana Multak
2,Murillo,Jessica,35,Dartling University,F,Dorothy,25005833,Jessica Murillo
3,Romanowski,Kandra,36,Dartling University,F,Genevieve,25002589,Kandra Romanowski
4,Hoffer,Katherine,37,Dartling University,F,Lynnette,25007185,Katherine Hoffer


In [25]:
student_load = student_load_df.to_dict('records')
#print(student_load)
try:
    sf.bulk.student__c.insert(student_load)
except Exception as e:
    print(e)

# Load Courses

In [98]:
data = pd.read_sql("SELECT * FROM course", conn)
data_load_df = data[['ID_Course','CourseCode','CourseName','CreditHours','BootCampCourse']].\
    rename(columns={'ID_Course':'ID_Course__c',
                    'CourseCode':'CourseCode__c',
                    'CourseName':'Name',
                    'CreditHours':'CreditHours__c',
                    'BootCampCourse':'BootCampCourse__c'})
data_load = data_load_df.to_dict('records')
print(f'There are {len(data_load)} records to load, record at index 0 is:\n {data_load[0]}')

There are 8 records to load, record at index 0 is:
 {'ID_Course__c': 1, 'CourseCode__c': 'BC-DATAVIZ', 'Name': 'Data Visualization and Analytics', 'CreditHours__c': 12, 'BootCampCourse__c': 1}


In [99]:
for record in data_load:
    try:
        #print(f'posting : {record}')
        sf.course__c.create(record)
    except Exception as e:
        print(e)

# load Staff

In [61]:
data = pd.read_sql("SELECT * FROM staff", conn)
data['FullName'] = data['FirstName'].str.cat(data['LastName'], sep=" ")
data_load_df = data.\
    rename(columns={'EmployeeID':'EmployeeID__c',
                    'ID_Staff':'ID_Staff__c',
                    'FirstName':'FirstName__c',
                    'FullName':'Name',
                    'LastName':'LastName__c',
                    'MiddleName':'MiddleName__c',
                    'BirthDate':'BirthDate__c'})
#data_load = data_load_df.to_dict('records')
print(f'There are {len(data_load)} records to load, record at index 0 is:\n {data_load[0]}')

There are 4 records to load, record at index 0 is:
 {'ID_Staff__c': 1, 'EmployeeID__c': '000184220', 'LastName__c': 'Wimberly', 'FirstName__c': 'Sam', 'MiddleName': 'Nico', 'BirthDate__c': datetime.date(1995, 7, 5), 'Name': 'Sam Wimberly'}


In [65]:
for index, row in data_load_df.iterrows():

    record = {
        'EmployeeID__c': row['EmployeeID__c'],
        'ID_Staff__c': row['ID_Staff__c'],
        'FirstName__c': row['FirstName__c'],
        'Name': row['Name'],
        'LastName__c': row['LastName__c'],
        'MiddleName__c': row['MiddleName__c'],
        'BirthDate__c': f"{row['BirthDate__c']}"
    }
    try:
        print(f'posting : {record}')
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

posting : {'EmployeeID__c': '000184220', 'ID_Staff__c': 1, 'FirstName__c': 'Sam', 'Name': 'Sam Wimberly', 'LastName__c': 'Wimberly', 'MiddleName__c': 'Nico', 'BirthDate__c': '1995-07-05'}
posting : {'EmployeeID__c': '000130109', 'ID_Staff__c': 2, 'FirstName__c': 'Gemini', 'Name': 'Gemini Sanford', 'LastName__c': 'Sanford', 'MiddleName__c': 'Blair', 'BirthDate__c': '1992-04-22'}
posting : {'EmployeeID__c': '000160655', 'ID_Staff__c': 3, 'FirstName__c': 'Dartanion', 'Name': 'Dartanion Williams', 'LastName__c': 'Williams', 'MiddleName__c': 'De Angelo', 'BirthDate__c': '1993-05-21'}
posting : {'EmployeeID__c': '000159108', 'ID_Staff__c': 4, 'FirstName__c': 'Sarah', 'Name': 'Sarah Popelka', 'LastName__c': 'Popelka', 'MiddleName__c': 'Nisan', 'BirthDate__c': '1994-06-07'}


# load classes

In [102]:
data = pd.read_sql("SELECT * FROM class", conn)
class_load_df = data.\
    rename(columns={'ID_Class':'ID_Class__c',
                    'ID_Course':'ID_Course__c',
                    'Section':'Name',
                    'StartDate':'StartDate__c',
                    'EndDate':'EndDate__c'})
#data_load = data_load_df.head(1).to_dict('records')
data = sf.query_all_iter("SELECT ID_Course__c, Id FROM course__c")
#data = sf.query("SELECT Id, ID_Course__c, FROM course__c")
course_lookup_lst = []

for row in data:
    rec = {
        'Course_Record_ID__c': row['Id'], # this is a critical line of code
        'ID_Course__c': int(row['ID_Course__c'])
    }
    course_lookup_lst.append(rec)
print(f'course_lookup_lst: {course_lookup_lst}')   
course_lookup_df = pd.DataFrame(course_lookup_lst)

data_load_df = pd.merge(class_load_df, course_lookup_df, on="ID_Course__c", how='left')

data_load = data_load_df.to_dict('records')
data_load[0]

course_lookup_lst: [{'Course_Record_ID__c': 'a043h00000MEbQtAAL', 'ID_Course__c': 4}, {'Course_Record_ID__c': 'a043h00000MEbRDAA1', 'ID_Course__c': 8}, {'Course_Record_ID__c': 'a043h00000MEbQoAAL', 'ID_Course__c': 3}, {'Course_Record_ID__c': 'a043h00000MEbQjAAL', 'ID_Course__c': 2}, {'Course_Record_ID__c': 'a043h00000MEbQeAAL', 'ID_Course__c': 1}, {'Course_Record_ID__c': 'a043h00000MEbQyAAL', 'ID_Course__c': 5}, {'Course_Record_ID__c': 'a043h00000MEbR8AAL', 'ID_Course__c': 7}, {'Course_Record_ID__c': 'a043h00000MEbR3AAL', 'ID_Course__c': 6}]


{'ID_Class__c': 1,
 'ID_Course__c': 1,
 'Name': 'GWU-ARL-DATA-PT-09-0',
 'StartDate__c': datetime.date(2020, 3, 16),
 'EndDate__c': datetime.date(2020, 3, 14),
 'Course_Record_ID__c': 'a043h00000MEbQeAAL'}

In [106]:
for index, row in data_load_df.iterrows():

    record = {
        'ID_Class__c': row['ID_Class__c'],
        'Name': row['Name'],
        'ID_Course__c': row['ID_Course__c'],
        'Course_Record_ID__c': row['Course_Record_ID__c'],
        'StartDate__c': f"{row['StartDate__c']}",
        'EndDate__c': f"{row['EndDate__c']}"
    }
    try:
        print(f'posting : {record}')
        sf.class__c.create(record)
    except Exception as e:
        print(e)

posting : {'ID_Class__c': 1, 'Name': 'GWU-ARL-DATA-PT-09-0', 'ID_Course__c': 1, 'Course_Record_ID__c': 'a043h00000MEbQeAAL', 'StartDate__c': '2020-03-16', 'EndDate__c': '2020-03-14'}
posting : {'ID_Class__c': 2, 'Name': 'GWDC201805DATA3', 'ID_Course__c': 1, 'Course_Record_ID__c': 'a043h00000MEbQeAAL', 'StartDate__c': '2018-05-15', 'EndDate__c': '2018-11-08'}
posting : {'ID_Class__c': 3, 'Name': 'GWARL201905WEB3', 'ID_Course__c': 2, 'Course_Record_ID__c': 'a043h00000MEbQjAAL', 'StartDate__c': '2019-05-14', 'EndDate__c': '2019-11-07'}
posting : {'ID_Class__c': 4, 'Name': 'GWARL201905UIUX3', 'ID_Course__c': 3, 'Course_Record_ID__c': 'a043h00000MEbQoAAL', 'StartDate__c': '2019-05-14', 'EndDate__c': '2019-11-07'}


# Load Staff assignment table

In [109]:
data = pd.read_sql("SELECT * FROM staffassignment", conn)
data

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [85]:
class_load_df = data.\
    rename(columns={'ID_Class':'ID_Class__c',
                    'ID_StaffAssignment':'ID_StaffAssignment__c',
                    'Role':'Role__c',
                    'StartDate':'StartDate__c',
                    'EndDate':'EndDate__c'})
#data_load = data_load_df.head(1).to_dict('records')
data = sf.query_all_iter("SELECT ID_Course__c, Id FROM class__c")
#data = sf.query("SELECT Id, ID_Course__c, FROM course__c")
course_lookup_lst = []

for row in data:
    rec = {
        'Course_Record_ID__c': row['Id'], # this is a critical line of code
        'ID_Course__c': int(row['ID_Course__c'])
    }
    course_lookup_lst.append(rec)
print(f'course_lookup_lst: {course_lookup_lst}')   
course_lookup_df = pd.DataFrame(course_lookup_lst)

data_load_df = pd.merge(class_load_df, course_lookup_df, on="ID_Course__c", how='left')
data_load = data_load_df.to_dict('records')
data_load[0]

[{'ID_Course': 1,
  'CourseCode__c': 'BC-DATAVIZ',
  'CourseName__c': 'Data Visualization and Analytics',
  'CreditHours__c': 12,
  'BootCampCourse__c': 1}]

## Convert DataFrame to a list of dictionaries 

This can help facilitate a bulk insert

In [68]:
data_load_df = data[['ID_Class','ID_Course','Section','StartDate','EndDate']].\
    rename(columns={'ID_Class':'ID_Class__c',
                    'ID_Course':'ID_Course__c',
                    'Section':'Section__c',
                    'StartDate':'StartDate__c',
                    'EndDate':'EndDate__c'})
data_load = data_load_df.head(1).to_dict('records')
data_load

[{'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': datetime.date(2020, 3, 14)}]

## Example of inserting rows individually

Loop through the DataFrame and assemble indiviudal dictionaries

In [76]:
for index, row in data_load_df.head(1).iterrows():

#     record = {
#         'LastName__c': row['LastName'],
#         'FirstName__c': row['FirstName'],
#         'Gender__c': row['Gender'],
#         'ID_Student__c':  row['ID_Student'],
#         'MiddleName__c': row['MiddleName'],
#         'Company__c': 'Dartling University'
#     }

#     record = {
#         'LastName__c': row['LastName'],
#         'FirstName__c': row['FirstName'],
#         'Gender__c': row['Gender'],
#         'ID_Student__c':  row['ID_Student'],
#         'MiddleName__c': row['MiddleName'],
#         'Company__c': 'Dartling University'
#     }
    
    record = {'ID_Class__c': row['ID_Class__c'],
          'ID_Course__c': row['ID_Course__c'],
          'Name': row['Section__c'],
          'StartDate__c': f"{row['StartDate__c']}",
          'EndDate__c': f"{row['EndDate__c']}"}
    
#        'StudentID__c':  row['StudentID'],
    try:
        print(f'posting : {record}')
        sf.cou__c.create(record)
    except Exception as e:
        print(e)

posting : {'ID_Class__c': 1, 'ID_Course__c': 1, 'Name': 'GWU-ARL-DATA-PT-09-0', 'StartDate__c': '2020-03-16', 'EndDate__c': '2020-03-14'}
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/class__c/. Response content: [{'message': 'ID_Course: id value of incorrect type: 1', 'errorCode': 'MALFORMED_ID', 'fields': ['ID_Course__c']}]


## Example of inserting rows in bulk

In [87]:
print(data_load)
try:
    sf.bulk.course__c.insert(data_load)
except Exception as e:
    print(e)

[{'ID_Course': 1, 'CourseCode__c': 'BC-DATAVIZ', 'CourseName__c': 'Data Visualization and Analytics', 'CreditHours__c': 12, 'BootCampCourse__c': 1}]
Malformed request https://na111.salesforce.com/services/async/42.0/job/7503h000003KTuQAAW/batch/7513h000003JEpWAAW/result. Response content: {'exceptionCode': 'InvalidBatch', 'exceptionMessage': 'Records not processed'}


In [36]:
try:
    data = sf.query("SELECT * FROM Student__c")
except Exception as e:
    print(e)

for row in data:
    print(row)

Malformed request https://na111.salesforce.com/services/data/v42.0/query/?q=SELECT+%2A+FROM+Student__c. Response content: [{'message': "\nSELECT * FROM Student__c\n      ^\nERROR at Row:1:Column:7\nunexpected token: '*'", 'errorCode': 'MALFORMED_QUERY'}]
totalSize
done
records


In [103]:
class_records = sf.query("SELECT Id FROM class__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
print(f'({len(recs_to_delete)}) records will be deleted')
sf.bulk.class__c.delete(recs_to_delete)

(8) records will be deleted


[{'success': True, 'created': False, 'id': 'a053h0000050uqxAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050ur2AAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050ur7AAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050urCAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050uvPAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050uvUAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050uvZAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a053h0000050uveAAA', 'errors': []}]